# Hoeveel te bestellen.

In [10]:
import pandas as pd
import math

def how_many_needed(item: str = ["Sensor", "Infuusset", "Reservoir"],
                 till_date: str = None,
                    nr_of_days: float = None,
                 stock: int = 0,
                 order: int = None,
                 margin: float = 0.2,
                 verbose: bool = True,
                 item_info: pd.DataFrame = None
                        ):
    """
    Calculate how many packages I need to order of a specific item, or whether the current order is sufficient.

    Parameters
    ----------
    item
        The diabetes item to check. One of ['Sensor', 'Infuusset', 'Reservoir'].
    till_date
        Until what date do I want to make sure to have enough supply? Give in the form 'YYYY-MM-DD'.
    nr_of_days
        Until what date do I want to make sure to have enough supply? Give in the form 'YYYY-MM-DD'.
    stock
        How many units do I've got in stock of this specific item?
    order
        If I want to check the current order amount: Fill in number of packages in the order.
    margin
        To prevent living on the edge: what's the margin to cover for item failings? As fraction.
    verbose
        Print output yes or no.
    item_info
        If I want to give a different dataframe than given in the function, give here.
        Must follow certain structure.
        Check the `item_info` dataframe in the function code for this structure.
    """

    # Check input.
    assert margin >= 0, "Margin should be non-negative."
    assert stock >= 0, "Stock should be non-negative."
    assert pd.to_datetime(till_date, errors='coerce') is not pd.NaT, "Invalid format for till_date. Should adhere to 'YYYY-MM-DD'."
    assert nr_of_days is not None or till_date is not None, "Give either the number of days you want to cover or until which date you want to be covered."

    # Background info.
    if item_info is None:
        item_info=pd.DataFrame({'Item':['Sensor', 'Infuusset', 'Reservoir'],
                        'Days per unit':[7, 3, 3],
                        'Units in package':[5,10,10],
                        'Official name':['Guardian Sensor 4', 'Mio Advance 6mm 60cm', 'Medtr reservoir 3ml']
                       })
    assert item_info is not None and not item_info.empty, "Item info DataFrame must be provided and non-empty."
    
    # Get the item's specifics.
    specs=item_info.loc[item_info['Item'] == item].iloc[0]
    days_per_unit, units_in_package, official_name = specs['Days per unit'], specs['Units in package'], specs['Official name']
    assert days_per_unit > 0 and units_in_package > 0, "Days per unit and units in package should be positive."
    
    # Whether to check the order or to calculate the minimal order.
    check_order = order is not None
    
    if verbose:
        print(f'-- Item: {item} --\n\nOfficial name: {official_name}.\n{stock} units in stock.')
        if check_order:
            print(f'Checking if an order with {order} package(s) is enough...')
        else:
            print('Calculating how many packages are needed...')
        lines='----------------------'
        print(lines)
    
    # Calculate how many days need to be covered.
    if till_date is not None:
        till_date, today = pd.to_datetime(till_date), pd.to_datetime('today')
        days_needed = (till_date-today).days*(1+margin)
        if verbose:
            print(f'Starting date: {today.date()}\nCovering until date: {till_date.date()}\n{round(days_needed,1)} days to be covered.\n{lines}')
    if nr_of_days is not None:
        days_needed = nr_of_days

    # Calculate.
    days_covered=stock*days_per_unit + (order*units_in_package*days_per_unit if check_order else 0)
    days_lacking=days_needed-days_covered
    more_to_order=math.ceil(days_lacking/days_per_unit/units_in_package)
    
    if verbose:
        print(f'{int(days_covered)} days are covered.')
        if days_lacking<=0:
            print(f'Stock covers enough days with {abs(int(days_lacking))} extra days.\n')
        else:
            print(f'{int(days_lacking)} days are not covered in this order.\n' if check_order else f'{int(days_lacking)} more days need to be covered.\n')
    
    # State how much more is needed.
    if check_order:
        verdict='Order is sufficient' if days_lacking<=0 else f'Not enough: {more_to_order} more package(s) needed.'
    else:
        verdict=f"{'Stock is sufficient' if days_lacking<=0 else f'Order must contain at least {more_to_order} package(s).'}"

    return verdict

In [11]:
how_many_needed(item='Infuusset', till_date='2024-04-02', stock=54)

-- Item: Infuusset --

Official name: Mio Advance 6mm 60cm.
54 units in stock.
Calculating how many packages are needed...
----------------------
Starting date: 2024-04-13
Covering until date: 2024-04-02
-14.4 days to be covered.
----------------------
162 days are covered.
Stock covers enough days with 176 extra days.



'Stock is sufficient'

In [6]:
how_many_needed(item='Infuusset', nr_of_days=7, stock=5)

-- Item: Infuusset --

Official name: Mio Advance 6mm 60cm.
5 units in stock.
Calculating how many packages are needed...
----------------------
15 days are covered.
Stock covers enough days with 8 extra days.



'Stock is sufficient'